## Log cleaner and combiner

After the timestamps have collected here I combine that log with the baseball savant log for the game. This attaches pitch outcome with an audio clip. This area is where I'll address tesseract as well (if there big I need to run it again). A couple rounds of error correction are performed.

First, I by eye remove obviously wrong tesseract pulls. Second, I fix a systematic issue that occurs which I will explain here. 

The last pitch of the inning is often immediately followed by commercials breaks. This does not give my program time to register that last pitch, leading to the last pitch of every halfing inning typically being missed. When that pitcher returns to the mound the next inning, this would give the program a chance to register the pitch count before the first pitch is thrown. The result is logging the last pitch count of the previous inning as having occured as the first pitch of the current inning. I pull up the savant log and remove these incorrect first pitches. 

In [1]:
#IMPORTS

import pandas as pd
import numpy as np

# MISSING INTEGER NUMBER SEQUENCE FINDER 
def missing_elements(L, start, end):
    if end - start <= 1: 
        if L[end] - L[start] > 1:
            yield from range(L[start] + 1, L[end])
        return

    index = start + (end - start) // 2

    # is the lower half consecutive?
    consecutive_low =  L[index] == L[start] + (index - start)
    if not consecutive_low:
        yield from missing_elements(L, start, index)

    # is the upper part consecutive?
    consecutive_high =  L[index] == L[end] - (end - index)
    if not consecutive_high:
        yield from missing_elements(L, index, end)

Next, I run the following routine to find all the duplicate and missing entries in the resulting join of the two tables. 

A duplicate means that something obscured the CV and had it register the same pitch count twice, and for some reason, the remove duplicate action did not remove one of them.

A missing entry means that the CV missed a pitch count or I deleted it because of one of the above reasons. This happens usually because of the end of the inning or a pitcher being pulled. In both cases the broadcast cuts to commercial quickly after the last pitch is thrown. Other cases are replays that eat up most of the pitch, the broadcast will jump back to the mound just before the pitch not giving the CV time to register. 

A log.txt is output that gives me a list of entries to correct and the log is kept for record purposes.

Be sure to run this cell only once!

In [3]:
#GAME NUMBER
a=6

path=f'E://HOU18/Playoffs 2017/games/game {a}/'
summary=pd.read_csv('E://HOU18/Playoffs 2017/2017playoff_games.csv')

#title=summary.iloc[a-1][8]
#print(f'game {a} - {title}')

n=0

file1=open(f'{path}initial_miss.txt','w')

df=pd.read_csv(f'{path}pitch timelog.csv')
df2=pd.read_csv(f'{path}savant log.csv')

new_df = pd.merge(df2, df,  how='left', left_on=['pitcher','pitch_count'], right_on = ['Pitcher','Pitch Count'])

#drop redundant columns
new_df.drop(['Pitch number','Pitcher','Pitch Count'],axis=1,inplace=True)

L = ['clips percentage missing\n',
    str(round((len(new_df['time'])-new_df['time'].count())/len(new_df['time'])*100,2))+'%\n\n'] 

# IF YOU UNCOMMENT ONLY WANT ASTRO HITTERS/OPPONENT PITCHERS
new_df=new_df[new_df['astro_batter']==1]
new_df.drop(['astro_batter'],axis=1,inplace=True)

file1.writelines(L)  

new_df['adjusted']=new_df['time']
new_df['start']=new_df['time']-20

new_df.to_csv(f'{path}pitch outcomes and time.csv',index=False)

# FIND ALL THE DUPLICATE ENTRIES

L=[]

x=new_df[new_df.duplicated(subset=['pitcher','pitch_count'],keep='first')]

file1.write('game pitch, pitcher, and pitch count that has a duplicate in initial join\n\n')

while n <  len(x['game_pitch'].values):
    
    L=str((x['game_pitch'].values[n],
    x['pitcher'].values[n],
    x['pitch_count'].values[n]))
    
    file1.writelines(L)
    file1.write('\n')
    n+=1

file1.write('\n')

L=[]

miss=list(new_df['game_pitch'].values)
miss=list(missing_elements(miss,0,len(miss)-1))

# FIND ALL THE MISSING ENTRIES
file1.write('game pitch, pitcher, and pitch count of missing pitches after initial join\n')
file1.write('\n')
for m in miss:
    L=str((df2['game_pitch'][m-1],df2['pitcher'][m-1],df2['pitch_count'][m-1]))
    file1.writelines(L)
    file1.write('\n')
file1.close()

Using the initial_miss.txt as a guide I fix the pitch timelog file by hand. Watching the video and pulling the timestamps manually. Typically this is for about 6% of the entries, better than having to do 100% by hand!

I'll continously run the cell until it returns 0% clips missing 

In [7]:
df=pd.read_csv(f'{path}pitch timelog.csv')
df2=pd.read_csv(f'{path}savant log.csv')

new_df = pd.merge(df2, df,  how='left', left_on=['pitcher','pitch_count'], right_on = ['Pitcher','Pitch Count'])
new_df.drop(['Pitch number','Pitcher','Pitch Count'],axis=1,inplace=True)

print('clips percentage missing')
print(str(round((len(new_df['time'])-new_df['time'].count())/len(new_df['time'])*100,2))+'%')

L = ['clips percentage missing\n',
    str(round((len(new_df['time'])-new_df['time'].count())/len(new_df['time'])*100,2))+'%\n']  

new_df=new_df[~new_df['time'].isnull()]

new_df['adjusted']=new_df['time']
new_df['start']=new_df['time']-20


new_df.to_csv(f'{path}pitch outcomes and time.csv',index=False)

clips percentage missing
0.0%


## COMBINING SUMMARY ROW WITH LOG

This fills each row of the pitch log with a set of columns from the summary file. This is a bit redundant, with each row stating the same set of info, but seeing ths format from Tony adams, I do like that he timestamps the youtube video for each pitch.

In [6]:
new_df=pd.read_csv(f'{path}pitch outcomes and time.csv')
summary=pd.read_csv('E://HOU18/Playoffs 2017/2017playoff_games.csv')

new_df['Game']=summary.iloc[a-1][0]
new_df['Date']=summary.iloc[a-1][1]
new_df['Opponent']=summary.iloc[a-1][2]
#new_df['Home']=summary.iloc[a-1][3]
new_df['Hou_Score']=summary.iloc[a-1][4]
new_df['Opp_Score']=summary.iloc[a-1][5]
new_df['Win']=summary.iloc[a-1][6]

new_df=new_df[['Game']+['Date']+['Opponent']+['Hou_Score']+['Opp_Score']+['Win']+['game_pitch']+['pitcher']+
                ['pitch_type']+['batter']+['pitch_count']+['plate_appearence']+['inning']+['result']+['pitch_velo_(mph)']+
                ['exit_velo_(mph)']+['launch_angle_(degrees)']+['distance_(ft)']+['xBA']+['time']+['adjusted']+['start']]

new_df['game_url']=summary.iloc[a-1][8]

n=0
stamp=[]
while n <  len(new_df['game_pitch'].values):
    stamp.append(f"{new_df['game_url'][n]}&t={new_df['start'][n]}")
    n+=1

new_df['timestamp']=stamp
new_df['pitch_url']=summary.iloc[a-1][7]

new_df.to_csv(f'{path}pitch outcomes and timetest.csv',index=False)